## IRS
(Import, Read, and Set Options)

In [51]:
import pandas as pd
from geopy.geocoders import GoogleV3
import numpy as np
import os

In [52]:
### Data from: https://www.fairhousingelements.org/he-status ###
# df = pd.read_csv('Website View HE status.csv')
df = pd.read_csv('compliance_data_with_cords.csv')

## IQ
### Instructions and Questions

<strong>Requirements:</strong>

- Map showing which cities are compliant with their housing element and which are not (two colors)

- Which cities had homes approved under the builder's element

- Pop-up/tooltip for cities to include data on housing element details
    - State mandated housing and affordable housing goals
    - What was accomplished during the last building cycle
- Pop-up/tooltip for builder's remedy
    - Details about approved homes, and who the developer is

## Get coordinates (done)

In [53]:
# %store -r google_maps_API_Key
# geolocator = GoogleV3(api_key=google_maps_API_Key)

In [54]:
# df['geo_location'] = df['Jurisdiction'] + ' ' + df['County'] + ' CA'

In [55]:
# df['display_location'] = df['Jurisdiction'].str.capitalize() + ' in ' + df['County']

In [56]:
# df['loc'] = df['geo_location'].apply(geolocator.geocode, timeout=10)

In [57]:
# df['loc']

In [58]:
# df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

In [59]:
# df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

In [60]:
df.columns

Index(['Unnamed: 0', 'Jurisdiction', 'County', 'HE Compliance', 'HE Due Date',
       'HE Status details', 'Adoption Date', 'COG', '6th Cycle Draft',
       'HCD Review Letter', 'Review Date', 'Population (2010)',
       'Median Income', 'geo_location', 'loc', 'point', 'lat', 'lon',
       'altitude', 'display_location'],
      dtype='object')

In [61]:
# df.to_csv('compliance_data_with_cords.csv')

## Make Map

In [62]:
### Make into two separate DFs ###
df_IN = df.loc[df['HE Compliance'] == 'IN']
df_OUT = df.loc[df['HE Compliance'] == 'OUT']

In [63]:
df.columns

Index(['Unnamed: 0', 'Jurisdiction', 'County', 'HE Compliance', 'HE Due Date',
       'HE Status details', 'Adoption Date', 'COG', '6th Cycle Draft',
       'HCD Review Letter', 'Review Date', 'Population (2010)',
       'Median Income', 'geo_location', 'loc', 'point', 'lat', 'lon',
       'altitude', 'display_location'],
      dtype='object')

In [64]:
def popup_html(row):
    i = row
    location = df['display_location'].iloc[i]
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>{}'''.format(location) + '''</strong><br>
    </html>
    '''
    return html


In [65]:
data = {'color':['green','red'],
       'compliance_status':['IN','OUT']}
df_color_lg = pd.DataFrame(data)

In [66]:
import folium
import branca

f = folium.Figure(width=750, height=750)
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=4)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'CA Housing Element Map')

for i in range(0,len(df_IN)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True), min_width=200, max_width=200)
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup,icon=folium.Icon(color='green')).add_to(m)
    
for i in range(0,len(df_OUT)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True), min_width=200, max_width=200)
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup,icon=folium.Icon(color='red')).add_to(m)
    
lgd_txt = '<span style="color: {col};">{txt}</span>'
    
for i, r in df_color_lg.iterrows():
    fg = folium.FeatureGroup(name=lgd_txt.format(txt=r['compliance_status'] + ' ' + '= ' + r['color'],col=r['color']))
    m.add_child(fg)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)

folium.TileLayer('Open Street Map',control=False).add_to(m)
    
folium.map.LayerControl('topleft', collapsed=False).add_to(m)

m

## Map URL Snagger
Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [67]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [68]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/CA_housing_element_map
